# Recommender Systems with DGL

## Introduction

Graph Neural Networks (GNN), as a methodology of learning representations on graphs, has gained much attention recently.  Various models such as Graph Convolutional Networks, GraphSAGE, etc. are proposed to obtain representations of whole graphs, or nodes on a single graph.

A primary goal of recommendation is to automatically make predictions about a user's interest, e.g. whether/how a user would interact with a set of items, given the interaction history of the user herself, as well as the histories of other users.  The user-item interaction can also be viewed as a bipartite graph, where users and items form two sets of nodes, and edges connecting them stands for interactions.  The problem can then be formulated as a *link-prediction* problem, where we try to predict whether an edge (of a given type) exists between two nodes.

Based on this intuition, the academia developed multiple new models for recommendation, including but not limited to:

* Geometric Learning Approaches
  * [Geometric Matrix Completion](https://papers.nips.cc/paper/5938-collaborative-filtering-with-graph-information-consistency-and-scalable-methods.pdf)
  * [Recurrent Multi-graph CNN](https://arxiv.org/pdf/1704.06803.pdf)
* Graph-convolutional Approaches
  * Models such as [R-GCN](https://arxiv.org/pdf/1703.06103.pdf) or [GraphSAGE](https://github.com/stellargraph/stellargraph/tree/develop/demos/link-prediction/hinsage) also apply.
  * [Graph Convolutional Matrix Completion](https://arxiv.org/abs/1706.02263)
  * [PinSage](https://arxiv.org/pdf/1806.01973.pdf)
  
In this hands-on tutorial, we will demonstrate how to write GraphSAGE in DGL + MXNet, and how to apply it in a recommender system setting.

## Dependencies

* Latest DGL release: `conda install -c dglteam dgl`
* `pandas`
* `stanfordnlp`
* `mxnet`
* `tqdm` for displaying the progress bar.

In [ ]:
import mxnet as mx
from mxnet import ndarray as nd, autograd, gluon
from mxnet.gluon import nn
import dgl
import dgl.function as FN
import numpy as np

np.random.seed(0)

## Loading data

In this tutorial, we focus on rating prediction on MovieLens-100K dataset.  The data comes from [MovieLens](http://files.grouplens.org/datasets/movielens/ml-1m.zip) and is shipped with the notebook already.

After loading and train-validation-test-splitting the dataset, we process the movie title into (padded) word-ID sequences, and other features into categorical variables (i.e. integers).  We then store them as node features on the graph.

Since user features and item features are different, we pad both types of features with zeros.

All of the above is encapsulated in `movielens.MovieLens` class for clarity of this notebook.

In [ ]:
import movielens
import stanfordnlp

# IMPORTANT!!!
# If you don't have stanfordnlp installed and the English models downloaded, please uncomment this statement
#stanfordnlp.download('en', force=True)

ml = movielens.MovieLens('ml-100k')

In [ ]:
g = ml.g
print('#vertices:', g.number_of_nodes())
print('#edges:', g.number_of_edges())

## See the features in the MovieLens dataset

The MovieLens dataset has some user features and movie features.

User features:
* age,
* gender,
* occupation,
* zip code,

Movie features:
* genre,
* year,
* title

We use one-hot encoding for "age", "gender", "occupation", "zip code" and "year". "genre" uses multi-hop encoding while "title" encodes the frequency of different words. We store "genre" and "title" in float32 dense matrices.

User nodes don't have features of movie nodes. These features on the user nodes are filled with 0. Similarly, movie nodes don't have features of user nodes and these features are filled with 0.

In [ ]:
print(g.ndata)

There is a node data "type" that indicates the node type in the bipartite graph. Nodes with "type=1" are user nodes and "type=0" are movie nodes.

In [ ]:
print('#user nodes:', mx.nd.sum(g.ndata['type']).asnumpy())
print('#movie nodes:', mx.nd.sum(g.ndata['type'] == 0).asnumpy())

# Recommendation model

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/rec_process.png" width="800">

Recommendation with graph neural networks has two steps:
* graph encoder: use graph neural networks to compute node embeddings.
* edge decoder: compute scores on edges with user embeddings and movie embeddings.

In [ ]:
class GNNRecommender(nn.Block):
    def __init__(self, encoder, decoder):
        super(GNNRecommender, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, G, users, items):
        h = self.encoder(G)
        h_users = h[users]
        h_items = h[items]
        return self.decoder(h_users, h_items, users, items)

## GraphSage encoder

The encoder does two things:
* generate the initial user and movie embeddings,
* run GraphSAGE layers on nodes multiple times to compute the final embeddings for rating prediction.

In [ ]:
class GraphSageEncoder(nn.Block):
    def __init__(self, embedding_size, n_layers, G):
        super(GraphSageEncoder, self).__init__()

        self.G = G
        self.user_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1).astype(np.int64)
        self.movie_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0).astype(np.int64)

        self.layers = nn.Sequential()
        for i in range(n_layers):
            self.layers.add(GraphSageLayer(embedding_size, G, self.user_nodes, self.movie_nodes))
        
        # One-hot encoding for each node.
        node_emb = nn.Embedding(G.number_of_nodes() + 1, embedding_size)
        self.user_emb = UserEmbedding(G, embedding_size, node_emb)
        self.movie_emb = MovieEmbedding(G, embedding_size, node_emb)

    def forward(self, G):
        # Generate embeddings on user nodes and movie nodes.
        assert G.number_of_nodes() == self.G.number_of_nodes()
        G.apply_nodes(lambda nodes: {'h': self.user_emb(nodes.data, self.user_nodes)}, self.user_nodes)
        G.apply_nodes(lambda nodes: {'h': self.movie_emb(nodes.data, self.movie_nodes)}, self.movie_nodes)

        for layer in self.layers:
            layer(G)

        return G.ndata['h']

## GraphSage model

We can now write a GraphSAGE layer.  In GraphSAGE, the node representation is updated with the representation in the previous layer as well as an aggregation (often mean) of "messages" sent from all neighboring nodes.

### Algorithm

The algorithm of a single GraphSAGE layer goes as follows for each node $v$:

1. $h_{\mathcal{N}(v)} \gets \mathtt{Sum}_{u \in \mathcal{N}(v)} h_{u}$
2. $h_{v} \gets \sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)}/d_{\mathcal{N}(v)})\right)$
3. $h_{v} \gets h_{v} / \lVert h_{v} \rVert_2$

### Slight modification on the original GraphSage model

In practice, the MovieLens dataset has two types of nodes: users and movies. We need to perform separate node update functions on the two types of nodes.

For the movie nodes,

$h_{m} \gets \sigma\left(W0 \cdot \mathtt{CONCAT}(h_m, h_{\mathcal{N}(m)} / d_{\mathcal{N}(m)})\right)$, 
$h_{m} \gets h_{m} / \lVert h_{m} \rVert_2$

For the user nodes,

$h_{u} \gets \sigma\left(W1 \cdot \mathtt{CONCAT}(h_u, h_{\mathcal{N}(u)} / d_{\mathcal{N}(u)})\right)$,
$h_{u} \gets h_{u} / \lVert h_{u} \rVert_2$

In [ ]:
class GraphSageLayer(nn.Block):
    def __init__(self, feature_size, G, user_nodes, movie_nodes):
        super(GraphSageLayer, self).__init__()

        self.feature_size = feature_size
        self.G = G
        self.user_nodes = user_nodes
        self.movie_nodes = movie_nodes

        self.user_update = GraphSageNodeUpdate(feature_size)
        self.movie_update = GraphSageNodeUpdate(feature_size)

        all_nodes = mx.nd.arange(G.number_of_nodes(), dtype=np.int64)
        self.deg = G.in_degrees(all_nodes).astype(np.float32)

    def forward(self, G):
        assert G.number_of_nodes() == self.G.number_of_nodes()
        G.ndata['deg'] = self.deg
        G.update_all(FN.copy_src('h', 'h'), FN.sum('h', 'h_agg'))
        G.apply_nodes(self.user_update, self.user_nodes)
        G.apply_nodes(self.movie_update, self.movie_nodes)

In [ ]:
class GraphSageNodeUpdate(nn.Block):
    def __init__(self, feature_size):
        super(GraphSageNodeUpdate, self).__init__()

        self.feature_size = feature_size
        self.W = nn.Dense(feature_size)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, nodes):
        # Node embedding from the previous layer.
        h = nodes.data['h']
        # Aggregation of the node embeddings in the neighborhood
        h_agg = nodes.data['h_agg']
        # Degree of the vertex.
        deg = nodes.data['deg'].expand_dims(1)
        h_concat = nd.concat(h, h_agg / nd.maximum(deg, 1e-6), dim=1)
        h_new = self.leaky_relu(self.W(h_concat))
        # Layer norm
        return {'h': h_new / nd.maximum(h_new.norm(axis=1, keepdims=True), 1e-6)}

## Compute node embeddings on the MovieLens dataset

User nodes and movie nodes have different sets of features. Thus, we need to generate embeddings differently.

User nodes have categorial features of "age", "gender", "occupation" and "zip code". These features are all one-hot encodings. In addition, we add one-hot encoding for every user node. To generate user embedding, we add all of these embeddings together.

In [ ]:
class UserEmbedding(nn.Block):
    def __init__(self, G, feature_size, node_emb):
        super(UserEmbedding, self).__init__()

        # Embedding matrices for one-hot encoding.
        self.emb_age = nn.Embedding(G.ndata['age'].max().asscalar() + 1,
                                    feature_size)
        self.emb_gender = nn.Embedding(G.ndata['gender'].max().asscalar() + 1,
                                       feature_size)
        self.emb_occupation = nn.Embedding(G.ndata['occupation'].max().asscalar() + 1,
                                           feature_size)
        self.emb_zip = nn.Embedding(G.ndata['zip'].max().asscalar() + 1,
                                    feature_size)

        # One-hot encoding for each node.
        self.node_emb = node_emb

    def forward(self, ndata, nid):
        h = self.node_emb(nid + 1)
        extra_repr = []
        extra_repr.append(self.emb_age(ndata['age']))
        extra_repr.append(self.emb_gender(ndata['gender']))
        extra_repr.append(self.emb_occupation(ndata['occupation']))
        extra_repr.append(self.emb_zip(ndata['zip']))
        return h + nd.stack(*extra_repr, axis=0).sum(axis=0)

Movie nodes "year", "genre" and "title". "year" is one-hot encoding, "genre" and "title" are stored in float32 dense matrices. Like user nodes, we add one-hot encoding to every movie node. To generate movie embedding, we add all of these embeddings together.

In [ ]:
class MovieEmbedding(nn.Block):
    def __init__(self, G, feature_size, node_emb):
        super(MovieEmbedding, self).__init__()
        self.emb_year = nn.Embedding(G.ndata['year'].max().asscalar() + 1,
                                     feature_size)

        # Linear projection for float32 features.
        seq = nn.Sequential()
        with seq.name_scope():
            seq.add(nn.Dense(feature_size))
            seq.add(nn.LeakyReLU(0.1))
        self.proj_genre = seq

        seq = nn.Sequential()
        with seq.name_scope():
            seq.add(nn.Dense(feature_size))
            seq.add(nn.LeakyReLU(0.1))
        self.proj_title = seq

        # One-hot encoding for each node.
        self.node_emb = node_emb

    def forward(self, ndata, nid):
        h = self.node_emb(nid + 1)
        extra_repr = []
        extra_repr.append(self.emb_year(ndata['year']))
        extra_repr.append(self.proj_genre(ndata['genre']))
        extra_repr.append(self.proj_title(ndata['title']))
        return h + nd.stack(*extra_repr, axis=0).sum(axis=0)

## Rating prediction

For recommendation, the rating on item $j$ by user $i$ is defined by $u_i^T v_j$.

In practice, recommendation models have user bias term and movie bias term: $u_i^T v_j + b_{u_i} + b_{v_j}$.

In [ ]:
class DotDecoder(nn.Block):
    def __init__(self, num_nodes):
        super(DotDecoder, self).__init__()
        
        with self.name_scope():
            self.biases = self.params.get(
                'node_biases',
                init=mx.init.Zero(),
                shape=(num_nodes+1,))
            
    def forward(self, h_users, h_items, users, items):
        user_biases = self.biases.data()[users+1]
        item_biases = self.biases.data()[items+1]
        return (h_users * h_items).sum(1) + user_biases + item_biases

## Get the training set

In [ ]:
g_train = ml.g_train
rating_train = g_train.edata['rating']
src_train, dst_train = g_train.all_edges()
print('#vertices:', g_train.number_of_nodes())
print('#edges:', g_train.number_of_edges())

## Get the testing edge set

In [ ]:
g = ml.g
eid_test = g.filter_edges(lambda edges: edges.data['test']).astype('int64')
src_test, dst_test = g.find_edges(eid_test)
rating_test = g.edges[eid_test].data['rating']
print('#testing edges:', len(eid_test))

## Run the model

In [ ]:
model = GNNRecommender(GraphSageEncoder(100, 1, g_train),
                       DotDecoder(g_train.number_of_nodes()))
model.collect_params().initialize(ctx=mx.cpu())
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.003, 'wd': 1e-5})

for epoch in range(200):
    # Training
    for _ in range(10):
        with mx.autograd.record():
            score = model(g_train, src_train, dst_train)
            loss = ((score - rating_train) ** 2).mean()
            loss.backward()
        trainer.step(1)

    # Testing
    h = model.encoder(g)
    # Compute test RMSE
    score = model.decoder(h[src_test], h[dst_test], src_test, dst_test)
    test_rmse = nd.sqrt(((score - rating_test) ** 2).mean())

    print('Training loss:', loss.asscalar(), 'Test RMSE:', test_rmse.asscalar())

In [ ]:
del g_train.ndata['h']
del g_train.ndata['h_agg']
del g.ndata['h']
del g.ndata['h_agg']